In [40]:
# Looking at population pyramids and IFR
# Method (and some of the code) based on Marc Bevand's work, see https://github.com/mbevand/covid19-age-stratified-ifr 

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)

import requests

import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

cmap = cm.get_cmap('Dark2',7)

import locale
import matplotlib.dates as mdates
# locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))


import os
import math

from datetime import date
import datetime

saveFigures = True
print('saveFigures is set to: '+str(saveFigures))
print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]
def plotWithMean(ax,x,y,meanWidth):
    firstLine = ax.plot(x,y,'.:',markersize=2,linewidth=0.5)
    firstColor = firstLine[0].get_color()
    ax.plot(rnTime(x,meanWidth),rnMean(y,meanWidth),color=firstColor)

saveFigures is set to: True
Done loading packages


In [41]:
# Pyramid data is from the United Nations: this file is a CSV export of the first sheet
# of "Population by Age Groups - Both Sexes" linked from:
# https://population.un.org/wpp/Download/Standard/Population/
# Direct link:
# https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/EXCEL_FILES/1_Population/WPP2019_POP_F07_1_POPULATION_BY_AGE_BOTH_SEXES.xlsx
file_pyramids = 'data/WPP2019_POP_F07_1_POPULATION_BY_AGE_BOTH_SEXES.csv'

maxage = 100
# Age groups defined in the CSV file
age_groups = [(0,4), (5,9), (10,14), (15,19), (20,24), (25,29), (30,34), (35,39), (40,44), (45,49), (50,54), (55,59), (60,64), (65,69), (70,74), (75,79), (80,84), (85,89), (90,94), (95,99), (100,maxage)]

# This will hold parsed pyramid data. Example to get the number of people in the
# age group 20-24 in France: pyramid['France'][(20,24)]
pyramid = {}

def ag2str(age_group):
    if age_group[1] == maxage:
        return f'{age_group[0]}+'
    return f'{age_group[0]}-{age_group[1]}'

def parse_pyramids():
    df = pd.read_csv(file_pyramids)
    # ignore labels as they don't contain any data
    df = df[df['Type'] != 'Label/Separator']
    # only take rows with data as of 2020
    df = df[df['Reference date (as of 1 July)'] == 2020]
    # only parse countries, world, and continents
    df = df[df['Type'].isin(('Country/Area', 'World', 'Region'))]
    # remove spaces used as thousands separators, and convert cell values to floats
    columns = [ag2str(x) for x in age_groups]
    for col in columns:
        df[col] = df[col].str.replace('\s+', '').astype(float)
    regions = list(df['Region, subregion, country or area *'])
    #regions = ('France',)
    for region in regions:
        pyramid[region] = {}
        df_region = df[df['Region, subregion, country or area *'] == region]
        for ag in age_groups:
            # values are in thousands
            pyramid[region][ag] = 1000 * float(df_region[ag2str(ag)])

In [42]:

# For a description of cdc_sympt, see the same variable name defined in covid_vs_flu.py
cdc_sympt = .33

# Various age-stratified IFR estimates
ifrs = [

        # Calculated from Spanish ENE-COVID study
        # (see calc_ifr.py)
        ('ENE-COVID', {
            (0,9):    0.003,
            (10,19):  0.004,
            (20,29):  0.015,
            (30,39):  0.030,
            (40,49):  0.064,
            (50,59):  0.213,
            (60,69):  0.718,
            (70,79):  2.384,
            (80,89):  8.466,
            (90,maxage): 12.497,
        }),

        # US CDC estimate as of 19 Mar 2021
        # https://www.cdc.gov/coronavirus/2019-ncov/hcp/planning-scenarios.html
        # (table 1)
        ('COVID: US CDC', {
            (0,17):   0.002,
            (18,49):  0.05,
            (50,64):  0.6,
            (65,maxage): 9.0,
        }),

        # Verity et al.
        # https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(20)30243-7/fulltext
        # (table 1)
        ('COVID: Verity', {
            (0,9):    0.00161,
            (10,19):  0.00695,
            (20,29):  0.0309,
            (30,39):  0.0844,
            (40,49):  0.161,
            (50,59):  0.595,
            (60,69):  1.93,
            (70,79):  4.28,
            (80,maxage): 7.80,
        }),

        # Levin et al.
        # https://link.springer.com/article/10.1007/s10654-020-00698-1
        # (table 3)
        ('COVID: Levin', {
            (0,34):   0.004,
            (35,44):  0.068,
            (45,54):  0.23,
            (55,64):  0.75,
            (65,74):  2.5,
            (75,84):  8.5,
            (85,maxage): 28.3,
        }),

        # Brazeau et al.
        # https://www.imperial.ac.uk/mrc-global-infectious-disease-analysis/covid-19/report-34-ifr/
        # (table 2, column "IFR (%) with Seroreversion")
        ('COVID: Brazeau', {
            (0,4):    0.00,
            (5,9):    0.01,
            (10,14):  0.01,
            (15,19):  0.02,
            (20,24):  0.02,
            (25,29):  0.04,
            (30,34):  0.06,
            (35,39):  0.09,
            (40,44):  0.15,
            (45,49):  0.23,
            (50,54):  0.36,
            (55,59):  0.57,
            (60,64):  0.89,
            (65,69):  1.39,
            (70,74):  2.17,
            (75,79):  3.39,
            (80,84):  5.30,
            (85,89):  8.28,
            (90,maxage): 16.19,
        }),

        # IFR for seasonal influenza
        # US CDC 2019-2020 influenza burden
        # https://www.cdc.gov/flu/about/burden/2019-2020.html
        ('Flu: US CDC', {
            (0,4):              254/4_291_677 * 100 * cdc_sympt,
            (5,17):             180/8_214_257 * 100 * cdc_sympt,
            (18,49):            2_669/15_325_708 * 100 * cdc_sympt,
            (50,64):            5_133/8_416_702 * 100 * cdc_sympt,
            (65,maxage):        13_673/1_946_161 * 100 * cdc_sympt,
        }),

]


In [43]:
parse_pyramids()
# pyramid

<ipython-input-41-80e8f36a4a51>:32: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace('\s+', '').astype(float)


In [44]:

def people_of_age(pyramid_region, age):
    # Returns the number of people of exact age 'age', given the provided age pyramid
    for ((a, b), n) in pyramid_region.items():
        if age in range(a, b + 1):
            return n / float(b - a + 1)


In [45]:
people_of_age(pyramid['Denmark'],80)

29400.0

In [46]:

def overall_ifr(pyramid_region, ifr_age_stratified):
    pop = 0
    deaths = 0
    for (age_group, ifr) in ifr_age_stratified.items():
        for age in range(age_group[0], age_group[1] + 1):
            pop += people_of_age(pyramid_region, age)
            deaths += people_of_age(pyramid_region, age) * ifr / 100.0
    assert pop == sum(pyramid_region.values())
    return 100.0 * deaths / pop

In [54]:
overall_ifr(pyramid_region=pyramid['Denmark'],ifr_age_stratified=ifrs[0][1])

0.7970310827145569

In [57]:
pyramid['Denmark']

{(0, 4): 309000.0,
 (5, 9): 297000.0,
 (10, 14): 337000.0,
 (15, 19): 339000.0,
 (20, 24): 374000.0,
 (25, 29): 402000.0,
 (30, 34): 355000.0,
 (35, 39): 317000.0,
 (40, 44): 361000.0,
 (45, 49): 377000.0,
 (50, 54): 422000.0,
 (55, 59): 388000.0,
 (60, 64): 346000.0,
 (65, 69): 309000.0,
 (70, 74): 351000.0,
 (75, 79): 235000.0,
 (80, 84): 147000.0,
 (85, 89): 78000.0,
 (90, 94): 36000.0,
 (95, 99): 10000.0,
 (100, 100): 1000.0}

In [ ]:
pyramid['Denmark'].items()

dict_items([((0, 4), 309000.0), ((5, 9), 297000.0), ((10, 14), 337000.0), ((15, 19), 339000.0), ((20, 24), 374000.0), ((25, 29), 402000.0), ((30, 34), 355000.0), ((35, 39), 317000.0), ((40, 44), 361000.0), ((45, 49), 377000.0), ((50, 54), 422000.0), ((55, 59), 388000.0), ((60, 64), 346000.0), ((65, 69), 309000.0), ((70, 74), 351000.0), ((75, 79), 235000.0), ((80, 84), 147000.0), ((85, 89), 78000.0), ((90, 94), 36000.0), ((95, 99), 10000.0), ((100, 100), 1000.0)])